### **运行环境**

In [ ]:
# pip install pandas numpy statsmodels spicy fastapi uvicorn
# 或
# pip install -r requirements.txt

---
### **结果查看**
> 在result目录下，有我已经测试的各个策略的选股结果和收益情况，欢迎借鉴。

---
### **使用方法**
#### 一、获取因子数据
> 支持使用内部定义的基本因子（持续更新），和用户自定义因子。自定义因子需要上传计算好的csv数据文件。

In [ ]:
import factors

"""
1.内部已经实现的因子
"""
vals = [
    factors.Market(),               # 市场因子 - 使用上证指数涨幅，不可用于预测
    factors.Size(),                 # 规模因子 - 使用市值计算
    factors.Value(),                # 价值因子 - 每股净资产/价格
    factors.Turnover(),             # 换手率因子 - 各股票每月换手率
    factors.Momentum(n=12,skip=1),  # 动量因子 - 过去 n 月涨跌幅， 跳过 skip 月
    factors.FScore(),               # 基本面因子 - 计算 FScore表示
    factors.MScore(),               # 造假因子 - 计算 MScore表示
    factors.Industry(),             # 行业哑变量 - 1/0表示
]

"""
2.自定义因子（详情见"./factors/factor.py"中的注释说明）
 - 在代码中进行如下定义
 - 在"./factors/factor_data"中传入你计算好的"factor_name.csv"
 - 支持的csv格式见详细说明
 - 注意文件名需要和"super().__init__"中传入的因子名称一样
"""
class MyFactor(factors.Factor):
    def __init__(self, param1=0, param2=0):
        super().__init__(f"factor_name", "N_T")

# 使用自定义因子
# vals.append(MyFactor())

#### 二、计算预期收益率、预期风险和求解权重
> 这里是因子投资实践的基本流程，即选出部分因子作为收益率预测模型，再选出部分因子作为风险预测模型，求解选股权重w使得预期收益最大、预期风险最小。在过程中求出的预期收益率，和最终求出的权重w都可用于选股。

In [ ]:
from factors import factor_lab

# 1.选择预测收益率需要的因子变量
vals1 = [
    # factors.Size(),
    # factors.Value(),
    factors.Turnover(),
    # factors.Momentum(12,1),
    factors.FScore(),
    # factors.MScore(),
    # factors.Industry(),
]

# 2.调用revenue方法，传入因子变量列表和用于预测的周期
pred = factor_lab.revenue(vals1, period=12)

# 3.将结果csv保存到指定路径
#  - wpath用于获取项目内设定的路径
pred.to_csv(factors.wpath('prediction1'))

# 4.选择预测风险需要的因子变量
#  - 注意：与预测收益率使用的因子不一定相同
vals2 = [
    # factors.Size(),
    # factors.Value(),
    factors.Turnover(),
    # factors.Momentum(12,1),
    factors.FScore(),
    # factors.MScore(),
    # factors.Industry(),
]

# 5.调用risk方法，传入因子变量列表和用于预测的周期
rsk = factor_lab.risk(vals2, period=36)

# 6.求解权重
#  - lam为风险厌恶系数，取值1-100，越大越保守
w = factor_lab.mvw(pred, rsk, lam=20)

# 7.将结果csv保存到指定路径
#  - wpath用于获取项目内设定的路径
w.to_csv(factors.wpath('w1'))

#### 三、策略回测
> 项目中实现的strategy类可支持自定义回测策略（详细说明见"strategies/strategy.py"中注释）。但是对于上一步得到的"prediction.csv"和"w.csv"，有专门适配的回测程序，只需要传入路径即可。

In [ ]:
import data_api
from strategies import StgPred, StgWeight
"""
1.基于预期收益率的策略回测
 - 每个时期等比例购入预期收益率排名前"max_num"的股票
"""
# 策略初始化
strategy = StgPred(cash=10000000,
                   datas=data_api.get_monthly_qfq('20161201', '20260201'),
                   wpath=factors.wpath('prediction1'), max_num=20, save=False)
# 回测执行
strategy.run()

In [ ]:
"""
2.基于权重的策略回测
 - 每个时期按照权重购买股票，依然可以设置最大持仓数量
"""
# 策略初始化
strategy = StgWeight(cash=10000000,
                     datas=data_api.get_monthly_qfq('20181201', '20260201'),
                     wpath=factors.wpath('w1'), max_num=20, save=False)

# 回测执行
strategy.run()

#### 四、实盘预测
> 在real_trading目录下实现当期选股预测